### Initialize Stuff

In [1]:
from jira import JIRA
from jira.client import ResultList
from jira.resources import Issue
from IPython.display import display, HTML, display_html
import pandas as pd
import numpy as np
import keyring  
import requests
import json
from requests.auth import HTTPBasicAuth
from bs4 import BeautifulSoup
from atlassian import Confluence
from dateutil import parser
from datetime import datetime
import datetime as DT

import xlwt
from xlwt import Workbook

### Initialize Variables

In [2]:
#get the authentication password from windows credential manager
JIRA_USER = "dapte"
JIRA_URL = "https://jira.opentext.com/"
CONFLUENCE_URL = 'https://confluence.opentext.com'

keyring_cred=keyring.get_password('keyring_cred', JIRA_USER)

#instantiate a jira object.
jira = JIRA(
    server = JIRA_URL,
    basic_auth=(JIRA_USER, keyring_cred)
)


confluence = Confluence(
    url= CONFLUENCE_URL,
    username=JIRA_USER, password=keyring_cred
)

userlist = []
userlist.append(JIRA_USER)

### Get all the user list to want the worklog hours for below

In [3]:
def validate(date_text):
    try:
        if date_text != datetime.strptime(date_text, "%Y/%m/%d").strftime('%Y/%m/%d'):
            raise ValueError
        return True
    except ValueError:
        return False
    
print("This utlity can be used to generate the worklog hours for you in the duration you specify . ( recent 1 week by default ). ")
fromDate =  input("[yyyy/mm/dd] Please enter the Date from:")
toDate = input("[yyyy/mm/dd] please enter the Date To:")

today = DT.date.today()
week_ago = today - DT.timedelta(days=7)

if validate(fromDate) and validate(toDate) :
    print("valid date")
else:
    print("invalid date entered , data will be collected for past 1 week from today ie: from :["+str(week_ago)+"] to ["+str(today)+"]")
    fromDate = week_ago
    toDate = today
#userlist = ["amareshk", "ssinghyadav","bbisht"]
#userlist = ["sbillakanti"]

#userlist = ["cmp","pthakekar","jtiwari","ynaikgau","rashmip","malathir","pbn","dnayak","pnatarajan","nadigs","rramanathan","sahua","spradhan","bkonapal","krangana","bparthasarat","vdash","bbisht","mmackowiak","sdobry","bhavyashreer","ssinghyadav","gfernandaz","ekaveripakam","amareshk","vmanchan","asengupt","aishwaryab","dapte"]
print(userlist)
#getUsername:

This utlity can be used to generate the worklog hours for you in the duration you specify . ( recent 1 week by default ). 


[yyyy/mm/dd] Please enter the Date from: 
[yyyy/mm/dd] please enter the Date To: 


invalid date entered , data will be collected for past 1 week from today ie: from :[2021-06-22] to [2021-06-29]
['dapte']


### Data Processing Step

In [4]:
#  JQL looks like this :  worklogAuthor in ( <username1> ...<usernameN>) and worklogDate >= '<fromDate>' and worklogDate <=  '<toDate>'

# TBD - maxResult capping to 500. can this go higher ? 
# TBD - error handling
#build the JQL based on the Userlist and the Date Range
jiralist="worklogAuthor in ( "
users=""
for user in userlist:
    users = users+user+", "
    
# strip the last comma and add the user in the JQL
jiralistJQL=jiralist+users[:-2]+") and worklogDate >= '"+str(fromDate)+"' and worklogDate <=  '"+str(toDate)+"'"
print(jiralistJQL)
issues =pd.Series(jira.search_issues(jiralistJQL, maxResults=500, fields= 'key', json_result = None),dtype=pd.StringDtype())

print(issues)

# https://your-domain.atlassian.net/rest/api/2/issue/{issueIdOrKey}/worklog
jiraReportDataFrame = pd.DataFrame(columns=["jira", "user","userid","timestamp","Hours","comment","labels"])

for issue in issues:
    isu = jira.issue(str(issue))
    labels = isu.fields.labels
    worklogs=isu.fields.worklog.worklogs
    for worklog in worklogs:
        if userlist.count(worklog.author.name) != 0 :
            jiraReportDataFrame = jiraReportDataFrame.append({
            'jira':isu,
            'user':worklog.author,
            'userid':worklog.author.name,
            'timestamp':worklog.created,
            'Hours':(worklog.timeSpentSeconds/3600),
                'comment':worklog.comment,
                'labels':labels
                
            },ignore_index=True)
    
table_user_util = pd.pivot_table(jiraReportDataFrame,index=["userid"],values=["Hours"],aggfunc=np.sum)
if table_user_util.empty:
    totalHours = 0
else:
    totalHours = round(table_user_util.iloc[0][0],2)

print("You have spent :"+ str(totalHours) +" Hours, Details have been commited to an output.xlsx file")
#print(jiraReportDataFrame)

jiraReportDataFrame.to_excel("output.xlsx") 

worklogAuthor in ( dapte) and worklogDate >= '2021-06-22' and worklogDate <=  '2021-06-29'
Series([], dtype: string)
You have spent :0 Hours, Details have been commited to an output.xlsx file
Empty DataFrame
Columns: [jira, user, userid, timestamp, Hours, comment, labels]
Index: []


In [5]:
##